# **Table of Contents** 


1.   Import Libraries
2.   Modelling
        *   Approach 1 - one for all, one for one, new devices prediction
        *   Approach 2 - one for all, one for one, new devices prediction
        *   Approach 3 - one for all, one for one, new devices prediction
        *   Approach 4 - one for all, one for one, new devices prediction
        *   Approach 5 - one for all, one for one, new devices prediction

3.   Submit


# **1. Import Libraries**

In [1]:
#'''
# Run with GPU 
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
#'''

Found GPU at: /device:GPU:0


In [2]:
validation=0

In [3]:
m=40
n=5

In [4]:
pip install -U layer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error as mae
import layer
from layer.decorators import dataset,model, pip_requirements

In [6]:
!pip install catboost
from catboost import CatBoostRegressor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
# Use newer xgboost version
!pip install xgboost==1.6.1
from xgboost import XGBRegressor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Log in to your Layer account

In [8]:
layer.login()

## Create a Layer project

In [9]:
layer.init("tomy4reel/aqssh-645h-wver")

Your Layer project is here: https://app.layer.ai/tomy4reel/aqssh-645h-wver

## Fetch the datasets as Pandas DataFrames

# **2. Modelling**

   - ## Approach 1

In [10]:
ss = layer.get_dataset("zindi/air-quality/datasets/sample_submission").to_pandas()
test_df = layer.get_dataset("zindi/air-quality/datasets/test").to_pandas()
train_df = layer.get_dataset("zindi/air-quality/datasets/train").to_pandas()
print(len(train_df),len(test_df))
target="pm2_5"
# Drop Correlared Cols
cc=['site_longitude','SulphurDioxide_SO2_column_number_density','Cloud_cloud_base_height','NitrogenDioxide_sensor_altitude','SulphurDioxide_sensor_azimuth_angle','SulphurDioxide_sensor_zenith_angle',
    'NitrogenDioxide_sensor_azimuth_angle','NitrogenDioxide_sensor_zenith_angle','Formaldehyde_sensor_azimuth_angle','Formaldehyde_sensor_zenith_angle','Ozone_sensor_azimuth_angle','Ozone_sensor_zenith_angle',
    'Cloud_sensor_azimuth_angle','Cloud_sensor_zenith_angle','CarbonMonoxide_sensor_azimuth_angle','CarbonMonoxide_sensor_azimuth_angle']
train_df.drop(cc,axis=1,inplace=True)
#Preprocess date
train_df['date']=pd.to_datetime(train_df['date'])
test_df['date']=pd.to_datetime(test_df['date'])
min_date=train_df['date'].min()
train_df['timestamp']=train_df['date']-min_date
test_df['timestamp']=test_df['date']-min_date
train_df['timestamp']=train_df['timestamp'].astype('int')/(24*60*60*(10**9))
test_df['timestamp']=test_df['timestamp'].astype('int')/(24*60*60*(10**9))
train_df['timestamp']=train_df['timestamp'].astype('int')
test_df['timestamp']=test_df['timestamp'].astype('int')
print(train_df.timestamp.min(),train_df.timestamp.max())
print(test_df.timestamp.min(),test_df.timestamp.max())
if validation==1:
    test_df=train_df[train_df.timestamp>=380]
    train_df=train_df[train_df.timestamp<380]
    test_df[target]=np.nan
    ss=test_df[['ID',target]]
    print(len(train_df),len(test_df))
# density features
density_amf = list(train_df.filter(regex='density_amf').columns)
density = ['SulphurDioxide_SO2_slant_column_number_density','SulphurDioxide_SO2_column_number_density_15km','CarbonMonoxide_CO_column_number_density','CarbonMonoxide_H2O_column_number_density',
 'NitrogenDioxide_NO2_column_number_density', 'NitrogenDioxide_tropospheric_NO2_column_number_density', 'NitrogenDioxide_stratospheric_NO2_column_number_density', 'NitrogenDioxide_NO2_slant_column_number_density', 'Formaldehyde_tropospheric_HCHO_column_number_density',
 'Formaldehyde_HCHO_slant_column_number_density','Ozone_O3_column_number_density','Ozone_O3_slant_column_number_density']
# Feature Clusters
Cloud = list(train_df.filter(regex='Cloud').columns)
Ozone = list(train_df.filter(regex='Ozone').columns)
Carbon = list(train_df.filter(regex='Carbon').columns)
Aerosol = list(train_df.filter(regex='Aerosol').columns)
Sulphur = list(train_df.filter(regex='Sulphur').columns)
Nitrogen = list(train_df.filter(regex='Nitrogen').columns)
Formaldehyde = list(train_df.filter(regex='Formaldehyde').columns)
Weather = ['temp_mean','humidity']
test_df[target]=np.nan
test_df=test_df[train_df.columns]
# new devices
missing=list(set(test_df.device.unique())-set(train_df.device.unique()))
print('number of new devices = ',len(missing))
train_df=train_df.append(test_df)
train_df['density_sum']=train_df[density].sum(axis=1)
train_df['density_amf_sum']=train_df[density_amf].sum(axis=1)
train_df['NO2_sum']=train_df.NitrogenDioxide_NO2_column_number_density+train_df.NitrogenDioxide_tropospheric_NO2_column_number_density
train_df=train_df.sort_values(['device','timestamp'])
# Feature generation
for col in Cloud+Ozone+Carbon+Aerosol+Sulphur+Nitrogen+Weather+Formaldehyde:
    train_df[col+'dev']=train_df.groupby('device')[col].transform('mean')
    train_df[col+'diff']=train_df[col]/train_df[col+'dev']
    train_df[col+'std']=train_df.groupby('device')[col].transform('std')
    b=3
    train_df[col+'rollin_mean']=train_df.groupby('device')[col].rolling(b,min_periods=b).mean().values
    for i in range(1,26,1):
        train_df[col+'change'+str(i)]=train_df.groupby('device')[col].shift(i)
        train_df[col+'change'+str(i)]=train_df[col]/train_df[col+'change'+str(i)]

le = LabelEncoder()
train_df = train_df.assign(device = le.fit_transform(train_df["device"]))
missing=le.transform(missing)
drop_cols=["pm2_5", "ID","date"]
test_df=train_df[train_df[target].isna()]
train_df=train_df[train_df[target].notna()]
cols=list(train_df.drop(drop_cols,axis=1).columns)
train_df=train_df.replace({np.inf:np.nan,(-1*np.inf):np.nan})
test_df=test_df.replace({np.inf:np.nan,(-1*np.inf):np.nan})
print('length of features = ',len(cols))

# One for All Prediction
#-----------------------------------------------
@model("model1all")
@pip_requirements(packages=["catboost"])
def train():
    import catboost
    from catboost import CatBoostRegressor
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    est=500
    random_state = 42
    layer.log({"est":est})
    layer.log({"random_state":random_state})
    model=CatBoostRegressor(silent=True,learning_rate=0.1, n_estimators=est).fit(train_df[cols],train_df[target])
    importance = model.get_feature_importance()
    importance_df = pd.DataFrame(importance, index=cols, columns=['Importance'])
    importance_df = importance_df.sort_values(by='Importance', ascending=False)
    importance_df = importance_df[importance_df['Importance'] > 0]
    importance_df = importance_df.head(10)
    plt.figure(figsize=(8,8))
    plt.xticks(rotation=60, fontsize = 20)
    sns.barplot(y=importance_df.index, x=importance_df['Importance'])
    layer.log({"Feature importance": plt.gcf()})
    return model
layer.run([train],debug=True)
model1all = layer.get_model("tomy4reel/aqssh-645h-wver/models/model1all").get_train()
submission = pd.DataFrame(data=test_df["ID"], columns = ["ID"])
submission["pm2_5"] = model1all.predict(test_df[cols])   
submission 

Output()

Output()

Output()

9923 4254


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: casting timedelta64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: casting timedelta64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, u

length of features =  1536


Output()

Output()

,ID,pm2_5
3557,ID_M1T3S50WVB,77.073039
1406,ID_35WTU72HHD,75.843380
925,ID_O7K2PDWG93,71.224603
3432,ID_N9WDA8G4OZ,67.153408
2205,ID_WFYVDUY1LY,84.775910
...,...,...
3325,ID_RK7GNMXWDG,51.947519
1766,ID_6IIMDQC260,46.028823
750,ID_8JPX2POPNL,60.153839
2549,ID_7TFB8NHLY9,60.831844


In [11]:
# One for One Prediction
#-----------------------------------------------
pred=pd.DataFrame()
df001=train_df.copy()
df002=test_df.copy()
model1=XGBRegressor(n_estimators=10)
train_df=train_df.append(test_df[train_df.columns])
train_df=train_df.sort_values(['device','timestamp'])
train_df2=train_df.copy()
for i in range(1,m,1):
    train_df['last_pm2_5']=train_df.groupby(['device'])[target].shift(i)
    for j in range(1,n,1):
        train_df['last_'+str(j)+'pm2_5']=train_df.groupby(['device'])[target].shift(i+j)
        train_df['last_'+str(j)+'pm2_5_trend']=train_df['last_'+str(j)+'pm2_5']/train_df['last_pm2_5']
    test_df=train_df[train_df[target].isna()]
    train_df=train_df[train_df[target].notna()]
    test_df=test_df[test_df['last_pm2_5'].notna()]
    train_df=train_df[train_df['last_pm2_5'].notna()]
    print(len(train_df),len(test_df),i)
    col=list(train_df.drop(drop_cols,axis=1).columns)
    test_df[target]=model1.fit(train_df[col],train_df[target]).predict(test_df[col])
    train_df=train_df2.copy()
    pred=pred.append(test_df[['ID',target]])
pred=pred.drop_duplicates('ID',keep='first')
submission=pred.append(submission)
submission[target]=submission.groupby('ID')[target].transform('mean')
submission=submission.drop_duplicates('ID',keep='first')
train_df=df001.copy()
test_df=df002.copy()
del df001, df002

9889 34 1
9855 68 2
9821 102 3
9787 136 4
9753 170 5
9719 204 6
9685 238 7
9651 272 8
9617 306 9
9583 340 10
9549 374 11
9515 408 12
9481 442 13
9447 476 14
9413 510 15
9379 544 16
9345 578 17
9311 612 18
9277 646 19
9243 680 20
9209 714 21
9175 748 22
9141 782 23
9107 816 24
9073 850 25
9039 884 26
9005 918 27
8971 952 28
8937 986 29
8903 1020 30
8869 1054 31
8835 1088 32
8801 1122 33
8767 1156 34
8733 1190 35
8699 1224 36
8665 1258 37
8631 1292 38
8597 1326 39


In [12]:
# New Devices Prediction
#-----------------------------------------------
test_df=test_df[test_df['device'].isin(missing)]
drop_cols=test_df.columns[test_df.isnull().all()].tolist()[1:]
cols=list(test_df[cols].drop(drop_cols,axis=1).columns)

@model("model1new")
@pip_requirements(packages=["catboost"])
def train():
    import catboost
    from catboost import CatBoostRegressor
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    est=200
    random_state = 42
    layer.log({"est":est})
    layer.log({"random_state":random_state})
    model=CatBoostRegressor(silent=True,learning_rate=0.1, n_estimators=est).fit(train_df[cols],train_df[target])
    importance = model.get_feature_importance()
    importance_df = pd.DataFrame(importance, index=cols, columns=['Importance'])
    importance_df = importance_df.sort_values(by='Importance', ascending=False)
    importance_df = importance_df[importance_df['Importance'] > 0]
    importance_df = importance_df.head(10)
    plt.figure(figsize=(8,8))
    plt.xticks(rotation=60, fontsize = 20)
    sns.barplot(y=importance_df.index, x=importance_df['Importance'])
    layer.log({"Feature importance": plt.gcf()})
    return model
layer.run([train],debug=True)
model1new = layer.get_model("tomy4reel/aqssh-645h-wver/models/model1new").get_train()
submission2 = pd.DataFrame(data=test_df["ID"], columns = ["ID"])
submission2["pm2_5"] = model1new.predict(test_df[cols])  
# replace predictions for new devices
submission=submission2.append(submission)
submission=submission.drop_duplicates('ID')
df1=pd.merge(ss[['ID']],submission,on='ID',how='left')
df1 

Output()

Output()

,ID,pm2_5
0,ID_UOH62J0XHX,45.684040
1,ID_4OPWSB0UHJ,96.491835
2,ID_3SLMNNG1Z3,58.086591
3,ID_MFUHTXYPM4,71.531910
4,ID_TY1DAND8ZP,69.430477
...,...,...
4249,ID_NYKGOYFBKZ,44.397399
4250,ID_IWO4EVLHSQ,76.812648
4251,ID_G1M1IMY9WU,61.134451
4252,ID_DXV65Q1QXI,53.813085


- ## Approach 2

In [13]:
# Reload datasets
test_df = layer.get_dataset("zindi/air-quality/datasets/test").to_pandas()
train_df = layer.get_dataset("zindi/air-quality/datasets/train").to_pandas()
# re-process data and create more features 

train_df.drop(cc,axis=1,inplace=True)
train_df['date']=pd.to_datetime(train_df['date'])
test_df['date']=pd.to_datetime(test_df['date'])
min_date=train_df['date'].min()
train_df['timestamp']=train_df['date']-min_date
test_df['timestamp']=test_df['date']-min_date
train_df['timestamp']=train_df['timestamp'].astype('int')/(24*60*60*(10**9))
test_df['timestamp']=test_df['timestamp'].astype('int')/(24*60*60*(10**9))
train_df['timestamp']=train_df['timestamp'].astype('int')
test_df['timestamp']=test_df['timestamp'].astype('int')
if validation==1:
    test_df=train_df[train_df.timestamp>=380]
    train_df=train_df[train_df.timestamp<380]
    test_df[target]=np.nan
    ss=test_df[['ID',target]]
    print(len(train_df),len(test_df))
density_amf = list(train_df.filter(regex='density_amf').columns)
density = ['SulphurDioxide_SO2_slant_column_number_density','SulphurDioxide_SO2_column_number_density_15km','CarbonMonoxide_CO_column_number_density','CarbonMonoxide_H2O_column_number_density',
 'NitrogenDioxide_NO2_column_number_density', 'NitrogenDioxide_tropospheric_NO2_column_number_density', 'NitrogenDioxide_stratospheric_NO2_column_number_density', 'NitrogenDioxide_NO2_slant_column_number_density', 'Formaldehyde_tropospheric_HCHO_column_number_density',
 'Formaldehyde_HCHO_slant_column_number_density','Ozone_O3_column_number_density','Ozone_O3_slant_column_number_density']
Cloud = list(train_df.filter(regex='Cloud').columns)
Ozone = list(train_df.filter(regex='Ozone').columns)
Carbon = list(train_df.filter(regex='Carbon').columns)
Aerosol = list(train_df.filter(regex='Aerosol').columns)
Sulphur = list(train_df.filter(regex='Sulphur').columns)
Nitrogen = list(train_df.filter(regex='Nitrogen').columns)
Formaldehyde = list(train_df.filter(regex='Formaldehyde').columns)
Weather = ['temp_mean','humidity']
test_df[target]=np.nan
test_df=test_df[train_df.columns]
# new devices
missing=list(set(test_df.device.unique())-set(train_df.device.unique()))
train_df=train_df.append(test_df)
train_df['density_sum']=train_df[density].sum(axis=1)
train_df['density_amf_sum']=train_df[density_amf].sum(axis=1)
train_df['NO2_sum']=train_df.NitrogenDioxide_NO2_column_number_density+train_df.NitrogenDioxide_tropospheric_NO2_column_number_density
train_df=train_df.sort_values(['device','timestamp'])
# feature generation
for col in Cloud+Ozone+Carbon+Aerosol+Sulphur+Nitrogen+Weather+Formaldehyde:
    train_df[col+'dev']=train_df.groupby('device')[col].transform('mean')
    train_df[col+'diff']=train_df[col]/train_df[col+'dev']
    train_df[col+'std']=train_df.groupby('device')[col].transform('std')
    #train_df[col+'skew']=train_df.groupby('device')[col].transform('skew')
    b=3
    train_df[col+'rollin_mean']=train_df.groupby('device')[col].rolling(b,min_periods=b).mean().values
    #train_df[col+'rollin_std']=train_df.groupby('device')[col].rolling(b,min_periods=b).std().values
    for i in range(1,35,1):
        train_df[col+'change'+str(i)]=train_df.groupby('device')[col].shift(i)
        train_df[col+'change'+str(i)]=train_df[col]/train_df[col+'change'+str(i)]
le = LabelEncoder()
train_df = train_df.assign(device = le.fit_transform(train_df["device"]))
missing=le.transform(missing)
drop_cols=["pm2_5", "ID","date"]
test_df=train_df[train_df[target].isna()]
train_df=train_df[train_df[target].notna()]
cols=list(train_df.drop(drop_cols,axis=1).columns)
train_df=train_df.replace({np.inf:np.nan,(-1*np.inf):np.nan})
test_df=test_df.replace({np.inf:np.nan,(-1*np.inf):np.nan})
print('length of features = ',len(cols))

# One for All Prediction
#-----------------------------------------------
@model("model2all")
@pip_requirements(packages=["catboost"])
def train():
    import catboost
    from catboost import CatBoostRegressor
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    est=500
    random_state = 42
    layer.log({"est":est})
    layer.log({"random_state":random_state})
    model=CatBoostRegressor(silent=True,learning_rate=0.1, n_estimators=est).fit(train_df[cols],train_df[target])
    importance = model.get_feature_importance()
    importance_df = pd.DataFrame(importance, index=cols, columns=['Importance'])
    importance_df = importance_df.sort_values(by='Importance', ascending=False)
    importance_df = importance_df[importance_df['Importance'] > 0]
    importance_df = importance_df.head(10)
    plt.figure(figsize=(8,8))
    plt.xticks(rotation=60, fontsize = 20)
    sns.barplot(y=importance_df.index, x=importance_df['Importance'])
    layer.log({"Feature importance": plt.gcf()})
    return model
layer.run([train],debug=True)
model2all = layer.get_model("tomy4reel/aqssh-645h-wver/models/model2all").get_train()
submission = pd.DataFrame(data=test_df["ID"], columns = ["ID"])
submission["pm2_5"] = model2all.predict(test_df[cols])  
submission  

Output()

Output()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: casting timedelta64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: casting timedelta64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which ha

length of features =  1995


Output()

Output()

,ID,pm2_5
3557,ID_M1T3S50WVB,78.833181
1406,ID_35WTU72HHD,75.477429
925,ID_O7K2PDWG93,69.638476
3432,ID_N9WDA8G4OZ,67.801012
2205,ID_WFYVDUY1LY,85.870084
...,...,...
3325,ID_RK7GNMXWDG,44.474359
1766,ID_6IIMDQC260,47.413532
750,ID_8JPX2POPNL,62.882649
2549,ID_7TFB8NHLY9,58.729734


In [14]:
# One for One Prediction
#-----------------------------------------------
pred=pd.DataFrame()
df001=train_df.copy()
df002=test_df.copy()
model2=XGBRegressor(n_estimators=10)
train_df=train_df.append(test_df[train_df.columns])
train_df=train_df.sort_values(['device','timestamp'])
train_df2=train_df.copy()
for i in range(1,m,1):
    train_df['last_pm2_5']=train_df.groupby(['device'])[target].shift(i)
    for j in range(1,n,1):
        train_df['last_'+str(j)+'pm2_5']=train_df.groupby(['device'])[target].shift(i+j)
        train_df['last_'+str(j)+'pm2_5_trend']=train_df['last_'+str(j)+'pm2_5']/train_df['last_pm2_5']
    test_df=train_df[train_df[target].isna()]
    train_df=train_df[train_df[target].notna()]
    test_df=test_df[test_df['last_pm2_5'].notna()]
    train_df=train_df[train_df['last_pm2_5'].notna()]
    print(len(train_df),len(test_df),i)
    col=list(train_df.drop(drop_cols,axis=1).columns)
    test_df[target]=model2.fit(train_df[col],train_df[target]).predict(test_df[col])
    train_df=train_df2.copy()
    pred=pred.append(test_df[['ID',target]])
pred=pred.drop_duplicates('ID',keep='first')
submission=pred.append(submission)
submission[target]=submission.groupby('ID')[target].transform('mean')
submission=submission.drop_duplicates('ID',keep='first')
train_df=df001.copy()
test_df=df002.copy()
del df001, df002

9889 34 1
9855 68 2
9821 102 3
9787 136 4
9753 170 5
9719 204 6
9685 238 7
9651 272 8
9617 306 9
9583 340 10
9549 374 11
9515 408 12
9481 442 13
9447 476 14
9413 510 15
9379 544 16
9345 578 17
9311 612 18
9277 646 19
9243 680 20
9209 714 21
9175 748 22
9141 782 23
9107 816 24
9073 850 25
9039 884 26
9005 918 27
8971 952 28
8937 986 29
8903 1020 30
8869 1054 31
8835 1088 32
8801 1122 33
8767 1156 34
8733 1190 35
8699 1224 36
8665 1258 37
8631 1292 38
8597 1326 39


In [15]:
# New Devices Prediction
#-----------------------------------------------
test_df=test_df[test_df['device'].isin(missing)]
drop_cols=test_df.columns[test_df.isnull().all()].tolist()[1:]
cols=list(test_df[cols].drop(drop_cols,axis=1).columns)

@model("model2new")
@pip_requirements(packages=["catboost"])
def train():
    import catboost
    from catboost import CatBoostRegressor
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    est=200
    random_state = 42
    layer.log({"est":est})
    layer.log({"random_state":random_state})
    model=CatBoostRegressor(silent=True,learning_rate=0.1, n_estimators=est).fit(train_df[cols],train_df[target])
    importance = model.get_feature_importance()
    importance_df = pd.DataFrame(importance, index=cols, columns=['Importance'])
    importance_df = importance_df.sort_values(by='Importance', ascending=False)
    importance_df = importance_df[importance_df['Importance'] > 0]
    importance_df = importance_df.head(10)
    plt.figure(figsize=(8,8))
    plt.xticks(rotation=60, fontsize = 20)
    sns.barplot(y=importance_df.index, x=importance_df['Importance'])
    layer.log({"Feature importance": plt.gcf()})
    return model
layer.run([train],debug=True)
model2new = layer.get_model("tomy4reel/aqssh-645h-wver/models/model2new").get_train()
submission2 = pd.DataFrame(data=test_df["ID"], columns = ["ID"])
submission2["pm2_5"] = model2new.predict(test_df[cols])    
# take mean 
submission=submission2.append(submission)
sm=submission.copy()
sm[target]=submission.groupby('ID')['pm2_5'].transform('mean')
df2=sm.drop_duplicates('ID')
df2

Output()

Output()

,ID,pm2_5
3492,ID_EF1OY5RDML,53.841963
4195,ID_V7OQFWQBJI,52.262942
982,ID_H5GZIE1XUI,42.812489
2330,ID_A8DVZ9U3EN,43.958055
2903,ID_RP7DMQ4CV9,45.246653
...,...,...
1485,ID_TTMFHMFJ91,60.234149
690,ID_T0N8DGLZJB,56.938791
2165,ID_3FU0SYNFLW,53.998520
1091,ID_PT0DLR6U76,54.484949


   - ## Approach 3

In [16]:
ss = layer.get_dataset("zindi/air-quality/datasets/sample_submission").to_pandas()
test_df = layer.get_dataset("zindi/air-quality/datasets/test").to_pandas()
train_df = layer.get_dataset("zindi/air-quality/datasets/train").to_pandas()
#Preprocess date
train_df['date']=pd.to_datetime(train_df['date'])
test_df['date']=pd.to_datetime(test_df['date'])

min_date=train_df['date'].min()
train_df['timestamp']=train_df['date']-min_date
test_df['timestamp']=test_df['date']-min_date
train_df['timestamp']=train_df['timestamp'].astype('int')/(24*60*60*(10**9))
test_df['timestamp']=test_df['timestamp'].astype('int')/(24*60*60*(10**9))
train_df['timestamp']=train_df['timestamp'].astype('int')
test_df['timestamp']=test_df['timestamp'].astype('int')
target="pm2_5"
if validation==1:
    test_df=train_df[train_df.timestamp>=380]
    ss=test_df[['ID']]
    scorer=test_df[['ID']]
    scorer['scorer']=test_df[target].values
    test_df[target]=np.nan
    train_df=train_df[train_df.timestamp<380]

new_devices=list(set(test_df.device.unique())-set(train_df.device.unique()))
train_df['year']=train_df['date'].astype(str).str[:4].astype('int')
train_df['month']=train_df['date'].astype(str).str[5:7].astype('int')
train_df['day']=train_df['date'].astype(str).str[8:10].astype('int')
test_df['year']=test_df['date'].astype(str).str[:4].astype('int')
test_df['month']=test_df['date'].astype(str).str[5:7].astype('int')
test_df['day']=test_df['date'].astype(str).str[8:10].astype('int')
c='NitrogenDioxide_tropospheric_NO2_column_number_density'
b='NitrogenDioxide_NO2_column_number_density'
a='CarbonMonoxide_CO_column_number_density'
train_df['d']=train_df[b]/train_df[c]
test_df['d']=test_df[b]/test_df[c]

cc=['SulphurDioxide_SO2_column_number_density','Cloud_cloud_base_height','NitrogenDioxide_sensor_altitude','SulphurDioxide_sensor_azimuth_angle','SulphurDioxide_sensor_zenith_angle',
    'NitrogenDioxide_sensor_azimuth_angle','NitrogenDioxide_sensor_zenith_angle','Formaldehyde_sensor_azimuth_angle','Formaldehyde_sensor_zenith_angle','Ozone_sensor_azimuth_angle','Ozone_sensor_zenith_angle',
    'Cloud_sensor_azimuth_angle','Cloud_sensor_zenith_angle','CarbonMonoxide_sensor_azimuth_angle','CarbonMonoxide_sensor_azimuth_angle']
# Drop Correlared Cols
train_df.drop(cc,axis=1,inplace=True)
# density features
density_amf = list(train_df.filter(regex='density_amf').columns)
density = ['SulphurDioxide_SO2_slant_column_number_density','SulphurDioxide_SO2_column_number_density_15km','CarbonMonoxide_CO_column_number_density','CarbonMonoxide_H2O_column_number_density',
 'NitrogenDioxide_NO2_column_number_density', 'NitrogenDioxide_tropospheric_NO2_column_number_density', 'NitrogenDioxide_stratospheric_NO2_column_number_density', 'NitrogenDioxide_NO2_slant_column_number_density', 'Formaldehyde_tropospheric_HCHO_column_number_density',
 'Formaldehyde_HCHO_slant_column_number_density','Ozone_O3_column_number_density','Ozone_O3_slant_column_number_density']
# Feature Clusters
Cloud = list(train_df.filter(regex='Cloud').columns)
Ozone = list(train_df.filter(regex='Ozone').columns)
Carbon = list(train_df.filter(regex='Carbon').columns)
Aerosol = list(train_df.filter(regex='Aerosol').columns)
Sulphur = list(train_df.filter(regex='Sulphur').columns)
Nitrogen = list(train_df.filter(regex='Nitrogen').columns)
Formaldehyde = list(train_df.filter(regex='Formaldehyde').columns)
Weather = ['temp_mean','humidity']

# Preprocess data
test_df[target]=np.nan
test_df=test_df[train_df.columns]
train_df=train_df.append(test_df)

train_df['month'+'_pm2_5']=train_df.groupby(['month'])[target].transform('mean')
#train_df['day'+'_pm2_5']=train_df.groupby(['day'])[target].transform('mean')
train_df['device'+'_pm2_5']=train_df.groupby(['device'])[target].transform('mean')
#train_df['year'+'_pm2_5']=train_df.groupby(['year'])[target].transform('mean')
train_df['site_latitude'+'_pm2_5']=train_df.groupby(['site_latitude'])[target].transform('mean')
#train_df['site_longitude'+'_pm2_5']=train_df.groupby(['site_longitude'])[target].transform('mean')
#train_df['location'+'_pm2_5']=train_df.groupby(['site_longitude','site_latitude'])[target].transform('mean')
train_df['density_sum']=train_df[density].sum(axis=1)
train_df['density_amf_sum']=train_df[density_amf].sum(axis=1)
train_df['NO2_sum']=train_df.NitrogenDioxide_NO2_column_number_density+train_df.NitrogenDioxide_tropospheric_NO2_column_number_density
train_df=train_df.sort_values(['device','timestamp'])
# Feature generation
for col in Cloud+Ozone+Carbon+Aerosol+Sulphur+Nitrogen+Weather+Formaldehyde:
    train_df[col+'dev']=train_df.groupby('device')[col].transform('mean')
    train_df[col+'diff']=train_df[col]/train_df[col+'dev']
    train_df[col+'std']=train_df.groupby('device')[col].transform('std')
    b=3
    train_df[col+'rollin_mean']=train_df.groupby('device')[col].rolling(b,min_periods=b).mean().values
    for i in range(1,26,1):
        train_df[col+'change'+str(i)]=train_df.groupby('device')[col].shift(i)
        train_df[col+'change'+str(i)]=train_df[col]/train_df[col+'change'+str(i)]
missing=train_df[train_df.device.isin(new_devices)]
le = LabelEncoder()
train_df = train_df.assign(device = le.fit_transform(train_df["device"]))
drop_cols=["pm2_5", "ID","date"]
test_df=train_df[train_df[target].isna()]
train_df=train_df[train_df[target].notna()]
cols=list(train_df.drop(drop_cols,axis=1).columns)
train_df=train_df.replace({np.inf:np.nan,(-1*np.inf):np.nan})
test_df=test_df.replace({np.inf:np.nan,(-1*np.inf):np.nan})
print('length of features = ',len(cols))

# One for All Prediction
#-----------------------------------------------

@model("model3all")
@pip_requirements(packages=["catboost"])
def train():
    import catboost
    from catboost import CatBoostRegressor
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    est=500
    random_state = 42
    layer.log({"est":est})
    layer.log({"random_state":random_state})
    model=CatBoostRegressor(silent=True,learning_rate=0.1, n_estimators=est).fit(train_df[cols],train_df[target])
    importance = model.get_feature_importance()
    importance_df = pd.DataFrame(importance, index=cols, columns=['Importance'])
    importance_df = importance_df.sort_values(by='Importance', ascending=False)
    importance_df = importance_df[importance_df['Importance'] > 0]
    importance_df = importance_df.head(10)
    plt.figure(figsize=(8,8))
    plt.xticks(rotation=60, fontsize = 20)
    sns.barplot(y=importance_df.index, x=importance_df['Importance'])
    layer.log({"Feature importance": plt.gcf()})
    return model
layer.run([train],debug=True)
model3all = layer.get_model("tomy4reel/aqssh-645h-wver/models/model3all").get_train()
submission = pd.DataFrame(data=test_df["ID"], columns = ["ID"])
submission["pm2_5"] = model3all.predict(test_df[cols]) 
submission


Output()

Output()

Output()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: casting timedelta64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: casting timedelta64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:75: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of cal

length of features =  1544


Output()

Output()

,ID,pm2_5
3557,ID_M1T3S50WVB,74.314712
1406,ID_35WTU72HHD,74.772778
925,ID_O7K2PDWG93,72.096713
3432,ID_N9WDA8G4OZ,71.801604
2205,ID_WFYVDUY1LY,85.820175
...,...,...
3325,ID_RK7GNMXWDG,36.540251
1766,ID_6IIMDQC260,35.904443
750,ID_8JPX2POPNL,45.819038
2549,ID_7TFB8NHLY9,36.943268


In [17]:
# One for One Approach
#-----------------------------------------------
pred=pd.DataFrame()
df001=train_df.copy()
df002=test_df.copy()
model3=XGBRegressor(n_estimators=10)
train_df=train_df.append(test_df[train_df.columns])
train_df=train_df.sort_values(['device','timestamp'])
train_df2=train_df.copy()
for i in range(1,m,1):
    train_df['last_pm2_5']=train_df.groupby(['device'])[target].shift(i)
    for j in range(1,n,1):
        train_df['last_'+str(j)+'pm2_5']=train_df.groupby(['device'])[target].shift(i+j)
        train_df['last_'+str(j)+'pm2_5_trend']=train_df['last_'+str(j)+'pm2_5']/train_df['last_pm2_5']
    test_df=train_df[train_df[target].isna()]
    train_df=train_df[train_df[target].notna()]
    test_df=test_df[test_df['last_pm2_5'].notna()]
    train_df=train_df[train_df['last_pm2_5'].notna()]
    print(len(train_df),len(test_df),i)
    col=list(train_df.drop(drop_cols,axis=1).columns)
    test_df[target]=model3.fit(train_df[col],train_df[target]).predict(test_df[col])
    train_df=train_df2.copy()
    pred=pred.append(test_df[['ID',target]])
pred=pred.drop_duplicates('ID',keep='first')
submission=pred.append(submission)
submission[target]=submission.groupby('ID')[target].transform('mean')
submission=submission.drop_duplicates('ID',keep='first')
train_df=df001.copy()
test_df=df002.copy()
del df001, df002

9889 34 1
9855 68 2
9821 102 3
9787 136 4
9753 170 5
9719 204 6
9685 238 7
9651 272 8
9617 306 9
9583 340 10
9549 374 11
9515 408 12
9481 442 13
9447 476 14
9413 510 15
9379 544 16
9345 578 17
9311 612 18
9277 646 19
9243 680 20
9209 714 21
9175 748 22
9141 782 23
9107 816 24
9073 850 25
9039 884 26
9005 918 27
8971 952 28
8937 986 29
8903 1020 30
8869 1054 31
8835 1088 32
8801 1122 33
8767 1156 34
8733 1190 35
8699 1224 36
8665 1258 37
8631 1292 38
8597 1326 39


In [18]:
# New Devices Prediction
#-----------------------------------------------
test_df=test_df[test_df['ID'].isin(list(missing['ID'].unique()))]
submission2=pd.DataFrame()
drop_cols=test_df.columns[test_df.isnull().all()].tolist()[1:]
cols=list(test_df[cols].drop(drop_cols,axis=1).columns)
@model("model3new")
@pip_requirements(packages=["catboost"])
def train():
    import catboost
    from catboost import CatBoostRegressor
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    est=200
    random_state = 42
    layer.log({"est":est})
    layer.log({"random_state":random_state})
    model=CatBoostRegressor(silent=True,learning_rate=0.1, n_estimators=est).fit(train_df[cols],train_df[target])
    importance = model.get_feature_importance()
    importance_df = pd.DataFrame(importance, index=cols, columns=['Importance'])
    importance_df = importance_df.sort_values(by='Importance', ascending=False)
    importance_df = importance_df[importance_df['Importance'] > 0]
    importance_df = importance_df.head(10)
    plt.figure(figsize=(8,8))
    plt.xticks(rotation=60, fontsize = 20)
    sns.barplot(y=importance_df.index, x=importance_df['Importance'])
    layer.log({"Feature importance": plt.gcf()})
    return model
layer.run([train],debug=True)
model3new = layer.get_model("tomy4reel/aqssh-645h-wver/models/model3new").get_train()
submission2 = pd.DataFrame(data=test_df["ID"], columns = ["ID"])
submission2["pm2_5"] = model3new.predict(test_df[cols])    
submission=submission2.append(submission)
submission=submission.drop_duplicates('ID')
# replace predictions for new devices
df3=pd.merge(ss[['ID']],submission,on='ID',how='left')
df3

Output()

Output()

,ID,pm2_5
0,ID_UOH62J0XHX,32.731282
1,ID_4OPWSB0UHJ,115.603892
2,ID_3SLMNNG1Z3,47.527696
3,ID_MFUHTXYPM4,72.837300
4,ID_TY1DAND8ZP,76.237748
...,...,...
4249,ID_NYKGOYFBKZ,45.554887
4250,ID_IWO4EVLHSQ,74.457139
4251,ID_G1M1IMY9WU,57.434548
4252,ID_DXV65Q1QXI,65.268846


- ## Approach 4

In [19]:
# Reload datasets
test_df = layer.get_dataset("zindi/air-quality/datasets/test").to_pandas()
train_df = layer.get_dataset("zindi/air-quality/datasets/train").to_pandas()
# re-process data and create more features 
train_df['date']=pd.to_datetime(train_df['date'])
test_df['date']=pd.to_datetime(test_df['date'])
min_date=train_df['date'].min()
train_df['timestamp']=train_df['date']-min_date
test_df['timestamp']=test_df['date']-min_date
train_df['timestamp']=train_df['timestamp'].astype('int')/(24*60*60*(10**9))
test_df['timestamp']=test_df['timestamp'].astype('int')/(24*60*60*(10**9))
train_df['timestamp']=train_df['timestamp'].astype('int')
test_df['timestamp']=test_df['timestamp'].astype('int')
target="pm2_5"
if validation==1:
    test_df=train_df[train_df.timestamp>=380]
    scorer=test_df[['ID']]
    scorer['scorer']=test_df[target].values
    test_df[target]=np.nan
    train_df=train_df[train_df.timestamp<380]

train_df['year']=train_df['date'].astype(str).str[:4].astype('int')
train_df['month']=train_df['date'].astype(str).str[5:7].astype('int')
train_df['day']=train_df['date'].astype(str).str[8:10].astype('int')
test_df['year']=test_df['date'].astype(str).str[:4].astype('int')
test_df['month']=test_df['date'].astype(str).str[5:7].astype('int')
test_df['day']=test_df['date'].astype(str).str[8:10].astype('int')
c='NitrogenDioxide_tropospheric_NO2_column_number_density'
b='NitrogenDioxide_NO2_column_number_density'
a='CarbonMonoxide_CO_column_number_density'
train_df['d']=train_df[b]/train_df[c]
test_df['d']=test_df[b]/test_df[c]

train_df.drop(cc,axis=1,inplace=True)
test_df[target]=np.nan
test_df=test_df[train_df.columns]
train_df=train_df.append(test_df)
train_df['month'+'_pm2_5']=train_df.groupby(['month'])[target].transform('mean')
#train_df['day'+'_pm2_5']=train_df.groupby(['day'])[target].transform('mean')
train_df['device'+'_pm2_5']=train_df.groupby(['device'])[target].transform('mean')
#train_df['year'+'_pm2_5']=train_df.groupby(['year'])[target].transform('mean')
train_df['site_latitude'+'_pm2_5']=train_df.groupby(['site_latitude'])[target].transform('mean')
#train_df['site_longitude'+'_pm2_5']=train_df.groupby(['site_longitude'])[target].transform('mean')
#train_df['location'+'_pm2_5']=train_df.groupby(['site_longitude','site_latitude'])[target].transform('mean')
train_df['density_sum']=train_df[density].sum(axis=1)
train_df['density_amf_sum']=train_df[density_amf].sum(axis=1)
train_df['NO2_sum']=train_df.NitrogenDioxide_NO2_column_number_density+train_df.NitrogenDioxide_tropospheric_NO2_column_number_density
train_df=train_df.sort_values(['device','timestamp'])
# feature generation
for col in Cloud+Ozone+Carbon+Aerosol+Sulphur+Nitrogen+Weather+Formaldehyde:
    train_df[col+'dev']=train_df.groupby('device')[col].transform('mean')
    train_df[col+'diff']=train_df[col]/train_df[col+'dev']
    train_df[col+'std']=train_df.groupby('device')[col].transform('std')
    #train_df[col+'skew']=train_df.groupby('device')[col].transform('skew')
    b=3
    train_df[col+'rollin_mean']=train_df.groupby('device')[col].rolling(b,min_periods=b).mean().values
    #train_df[col+'rollin_std']=train_df.groupby('device')[col].rolling(b,min_periods=b).std().values
    for i in range(1,35,1):
        train_df[col+'change'+str(i)]=train_df.groupby('device')[col].shift(i)
        train_df[col+'change'+str(i)]=train_df[col]/train_df[col+'change'+str(i)]
missing=train_df[train_df.device.isin(new_devices)]
le = LabelEncoder()
train_df = train_df.assign(device = le.fit_transform(train_df["device"]))
drop_cols=["pm2_5", "ID","date"]
test_df=train_df[train_df[target].isna()]
train_df=train_df[train_df[target].notna()]
cols=list(train_df.drop(drop_cols,axis=1).columns)
train_df=train_df.replace({np.inf:np.nan,(-1*np.inf):np.nan})
test_df=test_df.replace({np.inf:np.nan,(-1*np.inf):np.nan})
print('length of features = ',len(cols))

# One for All Prediction
#-----------------------------------------------

@model("model4all")
@pip_requirements(packages=["catboost"])
def train():
    import catboost
    from catboost import CatBoostRegressor
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    est=500
    random_state = 42
    layer.log({"est":est})
    layer.log({"random_state":random_state})
    model=CatBoostRegressor(silent=True,learning_rate=0.1, n_estimators=est).fit(train_df[cols],train_df[target])
    importance = model.get_feature_importance()
    importance_df = pd.DataFrame(importance, index=cols, columns=['Importance'])
    importance_df = importance_df.sort_values(by='Importance', ascending=False)
    importance_df = importance_df[importance_df['Importance'] > 0]
    importance_df = importance_df.head(10)
    plt.figure(figsize=(8,8))
    plt.xticks(rotation=60, fontsize = 20)
    sns.barplot(y=importance_df.index, x=importance_df['Importance'])
    layer.log({"Feature importance": plt.gcf()})
    return model
layer.run([train],debug=True)
model4all = layer.get_model("tomy4reel/aqssh-645h-wver/models/model4all").get_train()
submission = pd.DataFrame(data=test_df["ID"], columns = ["ID"])
submission["pm2_5"] = model4all.predict(test_df[cols]) 
submission   

Output()

Output()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: casting timedelta64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: casting timedelta64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: PerformanceWarning: DataFrame is highly fragmented.  Thi

length of features =  2003


Output()

Output()

,ID,pm2_5
3557,ID_M1T3S50WVB,78.083540
1406,ID_35WTU72HHD,72.884549
925,ID_O7K2PDWG93,78.262081
3432,ID_N9WDA8G4OZ,72.320999
2205,ID_WFYVDUY1LY,81.095007
...,...,...
3325,ID_RK7GNMXWDG,40.147772
1766,ID_6IIMDQC260,33.570919
750,ID_8JPX2POPNL,43.143023
2549,ID_7TFB8NHLY9,32.067789


In [20]:
# One for One Prediction
#-----------------------------------------------
pred=pd.DataFrame()
df001=train_df.copy()
df002=test_df.copy()
model4=XGBRegressor(n_estimators=10)
train_df=train_df.append(test_df[train_df.columns])
train_df=train_df.sort_values(['device','timestamp'])
train_df2=train_df.copy()
for i in range(1,m,1):
    train_df['last_pm2_5']=train_df.groupby(['device'])[target].shift(i)
    for j in range(1,n,1):
        train_df['last_'+str(j)+'pm2_5']=train_df.groupby(['device'])[target].shift(i+j)
        train_df['last_'+str(j)+'pm2_5_trend']=train_df['last_'+str(j)+'pm2_5']/train_df['last_pm2_5']
    test_df=train_df[train_df[target].isna()]
    train_df=train_df[train_df[target].notna()]
    test_df=test_df[test_df['last_pm2_5'].notna()]
    train_df=train_df[train_df['last_pm2_5'].notna()]
    print(len(train_df),len(test_df),i)
    col=list(train_df.drop(drop_cols,axis=1).columns)
    test_df[target]=model4.fit(train_df[col],train_df[target]).predict(test_df[col])
    train_df=train_df2.copy()
    pred=pred.append(test_df[['ID',target]])
    if validation==1:
        test_df=pd.merge(test_df[['ID',target]],scorer,on='ID',how='left')
        print(mae(test_df['scorer'],test_df[target]))
pred=pred.drop_duplicates('ID',keep='first')
submission=pred.append(submission)
submission[target]=submission.groupby('ID')[target].transform('mean')
submission=submission.drop_duplicates('ID',keep='first')
train_df=df001.copy()
test_df=df002.copy()
del df001, df002

9889 34 1
9855 68 2
9821 102 3
9787 136 4
9753 170 5
9719 204 6
9685 238 7
9651 272 8
9617 306 9
9583 340 10
9549 374 11
9515 408 12
9481 442 13
9447 476 14
9413 510 15
9379 544 16
9345 578 17
9311 612 18
9277 646 19
9243 680 20
9209 714 21
9175 748 22
9141 782 23
9107 816 24
9073 850 25
9039 884 26
9005 918 27
8971 952 28
8937 986 29
8903 1020 30
8869 1054 31
8835 1088 32
8801 1122 33
8767 1156 34
8733 1190 35
8699 1224 36
8665 1258 37
8631 1292 38
8597 1326 39


In [21]:
# New Devices Prediction
#-----------------------------------------------
test_df=test_df[test_df['ID'].isin(list(missing['ID'].unique()))]
submission2=pd.DataFrame()
drop_cols=test_df.columns[test_df.isnull().all()].tolist()[1:]
cols=list(test_df[cols].drop(drop_cols,axis=1).columns)
@model("model4new")
@pip_requirements(packages=["catboost"])
def train():
    import catboost
    from catboost import CatBoostRegressor
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    est=200
    random_state = 42
    layer.log({"est":est})
    layer.log({"random_state":random_state})
    model=CatBoostRegressor(silent=True,learning_rate=0.1, n_estimators=est).fit(train_df[cols],train_df[target])
    importance = model.get_feature_importance()
    importance_df = pd.DataFrame(importance, index=cols, columns=['Importance'])
    importance_df = importance_df.sort_values(by='Importance', ascending=False)
    importance_df = importance_df[importance_df['Importance'] > 0]
    importance_df = importance_df.head(10)
    plt.figure(figsize=(8,8))
    plt.xticks(rotation=60, fontsize = 20)
    sns.barplot(y=importance_df.index, x=importance_df['Importance'])
    layer.log({"Feature importance": plt.gcf()})
    return model
layer.run([train],debug=True)
model4new = layer.get_model("tomy4reel/aqssh-645h-wver/models/model4new").get_train()
submission2 = pd.DataFrame(data=test_df["ID"], columns = ["ID"])
submission2["pm2_5"] = model4new.predict(test_df[cols])   
# do not replace predictions for new devices with predictions from model4
# take mean instead
sm=submission2.append(submission)
submission=sm.copy()
submission[target]=sm.groupby('ID')['pm2_5'].transform('mean')
df4=submission.drop_duplicates('ID')
df4

Output()

Output()

,ID,pm2_5
3492,ID_EF1OY5RDML,50.724467
4195,ID_V7OQFWQBJI,42.390676
982,ID_H5GZIE1XUI,38.039749
2330,ID_A8DVZ9U3EN,36.928239
2903,ID_RP7DMQ4CV9,36.029801
...,...,...
1485,ID_TTMFHMFJ91,53.543479
690,ID_T0N8DGLZJB,55.839946
2165,ID_3FU0SYNFLW,59.778304
1091,ID_PT0DLR6U76,49.329259


   - ## Approach 5

In [22]:
ss = layer.get_dataset("zindi/air-quality/datasets/sample_submission").to_pandas()
test_df = layer.get_dataset("zindi/air-quality/datasets/test").to_pandas()
train_df = layer.get_dataset("zindi/air-quality/datasets/train").to_pandas()
print(len(train_df),len(test_df))
target="pm2_5"
new_devices=list(set(test_df.device.unique())-set(train_df.device.unique()))

# Drop Correlared Cols
cc=['SulphurDioxide_SO2_column_number_density','Cloud_cloud_base_height','NitrogenDioxide_sensor_altitude','SulphurDioxide_sensor_azimuth_angle','SulphurDioxide_sensor_zenith_angle',
    'NitrogenDioxide_sensor_azimuth_angle','NitrogenDioxide_sensor_zenith_angle','Formaldehyde_sensor_azimuth_angle','Formaldehyde_sensor_zenith_angle','Ozone_sensor_azimuth_angle','Ozone_sensor_zenith_angle',
    'Cloud_sensor_azimuth_angle','Cloud_sensor_zenith_angle','CarbonMonoxide_sensor_azimuth_angle','CarbonMonoxide_sensor_azimuth_angle']
train_df.drop(cc,axis=1,inplace=True)
#Preprocess date
train_df['date']=pd.to_datetime(train_df['date'])
test_df['date']=pd.to_datetime(test_df['date'])
min_date=train_df['date'].min()
train_df['timestamp']=train_df['date']-min_date
test_df['timestamp']=test_df['date']-min_date
train_df['timestamp']=train_df['timestamp'].astype('int')/(24*60*60*(10**9))
test_df['timestamp']=test_df['timestamp'].astype('int')/(24*60*60*(10**9))
train_df['timestamp']=train_df['timestamp'].astype('int')
test_df['timestamp']=test_df['timestamp'].astype('int')
print(train_df.timestamp.min(),train_df.timestamp.max())
print(test_df.timestamp.min(),test_df.timestamp.max())
if validation==1:
    test_df=train_df[train_df.timestamp>=380]
    train_df=train_df[train_df.timestamp<380]
    test_df[target]=np.nan
    ss=test_df[['ID',target]]
    print(len(train_df),len(test_df))
# density features
density_amf = list(train_df.filter(regex='density_amf').columns)
density = ['SulphurDioxide_SO2_slant_column_number_density','SulphurDioxide_SO2_column_number_density_15km','CarbonMonoxide_CO_column_number_density','CarbonMonoxide_H2O_column_number_density',
 'NitrogenDioxide_NO2_column_number_density', 'NitrogenDioxide_tropospheric_NO2_column_number_density', 'NitrogenDioxide_stratospheric_NO2_column_number_density', 'NitrogenDioxide_NO2_slant_column_number_density', 'Formaldehyde_tropospheric_HCHO_column_number_density',
 'Formaldehyde_HCHO_slant_column_number_density','Ozone_O3_column_number_density','Ozone_O3_slant_column_number_density']
# Feature Clusters
Cloud = list(train_df.filter(regex='Cloud').columns)
Ozone = list(train_df.filter(regex='Ozone').columns)
Carbon = list(train_df.filter(regex='Carbon').columns)
Aerosol = list(train_df.filter(regex='Aerosol').columns)
Sulphur = list(train_df.filter(regex='Sulphur').columns)
Nitrogen = list(train_df.filter(regex='Nitrogen').columns)
Formaldehyde = list(train_df.filter(regex='Formaldehyde').columns)
Weather = ['temp_mean','humidity']

# Preprocess data
test_df[target]=np.nan
test_df=test_df[train_df.columns]
train_df=train_df.append(test_df)

#train_df=train_df.sort_values(['device','timestamp'])
#train_df['no_of_dev']=train_df.groupby('timestamp')['device'].nunique()
#train_df['last_no_of_dev']=train_df.groupby('device')['no_of_dev'].shift(1)
#train_df['no_of_new_dev']=train_df['no_of_dev']-train_df['last_no_of_dev']
train_df['first_day']=train_df.groupby('device')['timestamp'].transform('min')
train_df['age_of_device']=train_df['timestamp']-train_df['first_day']
train_df['last_day']=train_df.groupby('device')['timestamp'].transform('max')
train_df['days_left']=train_df['last_day']-train_df['timestamp']

train_df['density_sum']=train_df[density].sum(axis=1)
train_df['density_amf_sum']=train_df[density_amf].sum(axis=1)
train_df['NO2_sum']=train_df.NitrogenDioxide_NO2_column_number_density+train_df.NitrogenDioxide_tropospheric_NO2_column_number_density
train_df=train_df.sort_values(['device','timestamp'])
# Feature generation
for col in Cloud+Ozone+Carbon+Aerosol+Sulphur+Nitrogen+Weather+Formaldehyde:
    train_df[col+'dev']=train_df.groupby('device')[col].transform('mean')
    train_df[col+'diff']=train_df[col]/train_df[col+'dev']
    train_df[col+'std']=train_df.groupby('device')[col].transform('std')
    b=3
    train_df[col+'rollin_mean']=train_df.groupby('device')[col].rolling(b,min_periods=b).mean().values
    for i in range(1,26,1):
        train_df[col+'change'+str(i)]=train_df.groupby('device')[col].shift(i)
        train_df[col+'change'+str(i)]=train_df[col]/train_df[col+'change'+str(i)]
for col in Cloud+Ozone+Carbon+Aerosol+Sulphur+Nitrogen+Weather+Formaldehyde:
    train_df[col+'dev2']=train_df.groupby('timestamp')[col].transform('mean')
    train_df[col+'diff2']=train_df[col]/train_df[col+'dev2']
    train_df[col+'std2']=train_df.groupby('timestamp')[col].transform('std')
    
# new devices
missing=train_df[train_df.device.isin(new_devices)]

le = LabelEncoder()
train_df = train_df.assign(device = le.fit_transform(train_df["device"]))
drop_cols=["pm2_5", "ID","date"]
test_df=train_df[train_df[target].isna()]
train_df=train_df[train_df[target].notna()]
cols=list(train_df.drop(drop_cols,axis=1).columns)
train_df=train_df.replace({np.inf:np.nan,(-1*np.inf):np.nan})
test_df=test_df.replace({np.inf:np.nan,(-1*np.inf):np.nan})
print('length of features = ',len(cols))
# One for All Prediction
#-----------------------------------------------
@model("model5all")
@pip_requirements(packages=["catboost"])
def train():
    import catboost
    from catboost import CatBoostRegressor
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    est=200
    random_state = 42
    layer.log({"est":est})
    layer.log({"random_state":random_state})
    model=CatBoostRegressor(silent=True,learning_rate=0.1, n_estimators=est).fit(train_df[cols],train_df[target])
    importance = model.get_feature_importance()
    importance_df = pd.DataFrame(importance, index=cols, columns=['Importance'])
    importance_df = importance_df.sort_values(by='Importance', ascending=False)
    importance_df = importance_df[importance_df['Importance'] > 0]
    importance_df = importance_df.head(10)
    plt.figure(figsize=(8,8))
    plt.xticks(rotation=60, fontsize = 20)
    sns.barplot(y=importance_df.index, x=importance_df['Importance'])
    layer.log({"Feature importance": plt.gcf()})
    return model
layer.run([train],debug=True)
model5all = layer.get_model("tomy4reel/aqssh-645h-wver/models/model5all").get_train()
submission = pd.DataFrame(data=test_df["ID"], columns = ["ID"])
submission["pm2_5"] = model5all.predict(test_df[cols]) 
submission   

Output()

Output()

Output()

9923 4254
0 525
525 658


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: casting timedelta64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: casting timedelta64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:66: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all 

length of features =  1694


Output()

Output()

,ID,pm2_5
3557,ID_M1T3S50WVB,73.923428
1406,ID_35WTU72HHD,71.182961
925,ID_O7K2PDWG93,77.842260
3432,ID_N9WDA8G4OZ,70.722704
2205,ID_WFYVDUY1LY,79.585122
...,...,...
3325,ID_RK7GNMXWDG,56.878590
1766,ID_6IIMDQC260,64.655453
750,ID_8JPX2POPNL,58.169381
2549,ID_7TFB8NHLY9,53.871476


In [23]:
# One for One Prediction
#-----------------------------------------------
pred=pd.DataFrame()
df001=train_df.copy()
df002=test_df.copy()
est=200
model5=CatBoostRegressor(silent=True,learning_rate=0.1, n_estimators=est)
train_df=train_df.append(test_df[train_df.columns])
train_df=train_df.sort_values(['device','timestamp'])
train_df2=train_df.copy()
for i in range(1,m,1):
    train_df['last_pm2_5']=train_df.groupby(['device'])[target].shift(i)
    for j in range(1,n,1):
        train_df['last_'+str(j)+'pm2_5']=train_df.groupby(['device'])[target].shift(i+j)
        train_df['last_'+str(j)+'pm2_5_trend']=train_df['last_'+str(j)+'pm2_5']/train_df['last_pm2_5']
    test_df=train_df[train_df[target].isna()]
    train_df=train_df[train_df[target].notna()]
    test_df=test_df[test_df['last_pm2_5'].notna()]
    train_df=train_df[train_df['last_pm2_5'].notna()]
    print(len(train_df),len(test_df),i)
    col=list(train_df.drop(drop_cols,axis=1).columns)
    test_df[target]=model5.fit(train_df[col],train_df[target]).predict(test_df[col])
    train_df=train_df2.copy()
    pred=pred.append(test_df[['ID',target]])
    if validation==1:
        test_df=pd.merge(test_df[['ID',target]],scorer,on='ID',how='left')
        print(mae(test_df['scorer'],test_df[target]))
pred=pred.drop_duplicates('ID',keep='first')
submission=pred.append(submission)
#submission[target]=submission.groupby('ID')[target].transform('mean')
submission=submission.drop_duplicates('ID',keep='first')
train_df=df001.copy()
test_df=df002.copy()
del df001, df002

9889 34 1
9855 68 2
9821 102 3
9787 136 4
9753 170 5
9719 204 6
9685 238 7
9651 272 8
9617 306 9
9583 340 10
9549 374 11
9515 408 12
9481 442 13
9447 476 14
9413 510 15
9379 544 16
9345 578 17
9311 612 18
9277 646 19
9243 680 20
9209 714 21
9175 748 22
9141 782 23
9107 816 24
9073 850 25
9039 884 26
9005 918 27
8971 952 28
8937 986 29
8903 1020 30
8869 1054 31
8835 1088 32
8801 1122 33
8767 1156 34
8733 1190 35
8699 1224 36
8665 1258 37
8631 1292 38
8597 1326 39


In [24]:
# New Devices Prediction
#-----------------------------------------------
test_df=test_df[test_df['ID'].isin(list(missing['ID'].unique()))]
submission2=pd.DataFrame()
drop_cols=test_df.columns[test_df.isnull().all()].tolist()[1:]
cols=list(test_df[cols].drop(drop_cols,axis=1).columns)
@model("model5new")
@pip_requirements(packages=["catboost"])
def train():
    import catboost
    from catboost import CatBoostRegressor
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    est=200
    random_state = 42
    layer.log({"est":est})
    layer.log({"random_state":random_state})
    model=CatBoostRegressor(silent=True,learning_rate=0.1, n_estimators=est).fit(train_df[cols],train_df[target])
    importance = model.get_feature_importance()
    importance_df = pd.DataFrame(importance, index=cols, columns=['Importance'])
    importance_df = importance_df.sort_values(by='Importance', ascending=False)
    importance_df = importance_df[importance_df['Importance'] > 0]
    importance_df = importance_df.head(10)
    plt.figure(figsize=(8,8))
    plt.xticks(rotation=60, fontsize = 20)
    sns.barplot(y=importance_df.index, x=importance_df['Importance'])
    layer.log({"Feature importance": plt.gcf()})
    return model
layer.run([train],debug=True)
model5new = layer.get_model("tomy4reel/aqssh-645h-wver/models/model5new").get_train()
submission2 = pd.DataFrame(data=test_df["ID"], columns = ["ID"])
submission2["pm2_5"] = model5new.predict(test_df[cols])    
submission=submission2.append(submission)
submission=submission.drop_duplicates('ID')
# replace predictions for new devices 
df5=pd.merge(ss[['ID']],submission,on='ID',how='left')
df5

Output()

Output()

,ID,pm2_5
0,ID_UOH62J0XHX,52.633002
1,ID_4OPWSB0UHJ,93.021880
2,ID_3SLMNNG1Z3,54.317341
3,ID_MFUHTXYPM4,84.976998
4,ID_TY1DAND8ZP,80.419835
...,...,...
4249,ID_NYKGOYFBKZ,57.571389
4250,ID_IWO4EVLHSQ,65.104465
4251,ID_G1M1IMY9WU,57.078758
4252,ID_DXV65Q1QXI,68.697898


# **3. Submit**

In [25]:
# blend predictions
df=df1.copy()
df2=pd.merge(df1[[df1.columns[0]]],df2,on=df1.columns[0],how='left')
df3=pd.merge(df1[[df1.columns[0]]],df3,on=df1.columns[0],how='left')
df4=pd.merge(df1[[df1.columns[0]]],df4,on=df1.columns[0],how='left')
df5=pd.merge(df1[[df1.columns[0]]],df5,on=df1.columns[0],how='left')
for col in df1.columns[1:]:
    print(col)
    df[col]=(df1[[col]]*0.1225)+(df2[[col]]*0.2275)+(df3[[col]]*0.1225)+(df4[[col]]*0.2275)+(df5[[col]]*0.3)
if validation==0:
  df.to_csv('prediction.csv',index=False)
elif validation==1:
  scorer = layer.get_dataset("zindi/air-quality/datasets/train").to_pandas()
  scorer=pd.merge(df,scorer[['ID',target]],on='ID',how='left')
  acc=mae(scorer['pm2_5_y'], scorer['pm2_5_x'])
  print(acc)

pm2_5


In [26]:
df

,ID,pm2_5
0,ID_UOH62J0XHX,45.668597
1,ID_4OPWSB0UHJ,101.083434
2,ID_3SLMNNG1Z3,52.105219
3,ID_MFUHTXYPM4,74.014803
4,ID_TY1DAND8ZP,75.314358
...,...,...
4249,ID_NYKGOYFBKZ,47.712011
4250,ID_IWO4EVLHSQ,71.101942
4251,ID_G1M1IMY9WU,58.919457
4252,ID_DXV65Q1QXI,62.785765
